1.  Опишите  общую  схему  поиска  изображений,  включая  роль  дескрипторов 
изображений  и  методы  ранжирования  результатов.  Как  семантическое 
хеширование и алгоритм GIST используются для улучшения эффективности и 
точности  поиска?  В  чем  заключается  вклад  этих  технологий  в  улучшение 
доступа к большим коллекциям изображений? 
2.  Реализуйте алгоритм сшивания изображений для создания панорамы из серии 
фотографий. 
  


Извлечение признаков – представление изображения в числовом виде с помощью дескрипторов.
Индексирование – хранение и организация этих представлений для быстрого поиска.
Ранжирование результатов – сортировка изображений по степени сходства с запросом.

Дескрипторы – это числовые представления изображений, извлекаемые с помощью алгоритмов компьютерного зрения.

Глубокая нейросеть обучается так, чтобы похожие изображения получали близкие хеш-коды.
Поиск по хеш-кодам выполняется за O(1) операций, что намного быстрее, чем сравнение полных дескрипторов.

GIST:
Разделяет изображение на регионы.
Применяет фильтры для извлечения информации о текстуре и ориентации.
Формирует компактный вектор признаков (~512 значений).

In [ ]:
import cv2
import numpy as np


def create_panorama(image1_path, image2_path, output_path="panorama.jpg"):
    # Загружаем изображения
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)

    # Инициализируем ORB детектор
    orb = cv2.ORB_create()

    # Находим ключевые точки и дескрипторы с помощью ORB
    keypoints1, descriptors1 = orb.detectAndCompute(image1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(image2, None)

    # Создаем объект BFMatcher и совершаем сопоставление дескрипторов
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)

    # Сортируем сопоставления по расстоянию (лучшие сопоставления первые)
    matches = sorted(matches, key=lambda x: x.distance)

    # Извлекаем координаты соответствующих ключевых точек для сопоставленных дескрипторов
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    # Находим матрицу гомографии
    H, _ = cv2.findHomography(points1, points2, cv2.RANSAC)

    # Применяем преобразование гомографии к изображению 1
    height, width, channels = image2.shape
    panorama = cv2.warpPerspective(image1, H, (width * 2, height))

    # Копируем изображение 2 в панорамное изображение
    panorama[0 : image2.shape[0], 0 : image2.shape[1]] = image2

    # Сохраняем панорамное изображение
    cv2.imwrite(output_path, panorama)

    return image2


# Пример использования:
create_panorama("./data/panorama02.jpg", "./data/panorama01.jpg", "./data/output_panorama.jpg")

array([[[180, 101,  52],
        [181, 102,  53],
        [183, 104,  55],
        ...,
        [215, 143, 101],
        [215, 143, 101],
        [211, 139,  99]],

       [[184, 105,  56],
        [186, 107,  58],
        [187, 108,  59],
        ...,
        [212, 140, 100],
        [212, 140, 100],
        [214, 142, 102]],

       [[186, 107,  58],
        [188, 109,  60],
        [189, 110,  61],
        ...,
        [216, 144, 104],
        [216, 144, 104],
        [217, 145, 105]],

       ...,

       [[ 74,  31,  16],
        [ 74,  31,  16],
        [ 74,  31,  16],
        ...,
        [135,  71,  47],
        [135,  71,  47],
        [135,  71,  47]],

       [[ 74,  31,  16],
        [ 74,  31,  16],
        [ 74,  31,  16],
        ...,
        [135,  71,  47],
        [135,  71,  47],
        [135,  71,  47]],

       [[ 74,  31,  16],
        [ 74,  31,  16],
        [ 74,  31,  16],
        ...,
        [135,  71,  47],
        [135,  71,  47],
        [135,  71,  47]]